# κ‑Hamiltonian Experimental Signature — Cross‑Domain Fit (2025-10-28T03:50:08)

**Goal:** Demonstrate that a **helical (κ‑adjusted)** oscillator model beats a standard sinusoid on **AIC/BIC** using *the same* dataset.

You can supply **any CSV** with columns `(t, y)` (e.g., PhysioNet HRV, NMR precession, macro cycles).

In [ ]:
import numpy as np, pandas as pd
from math import sin, cos, pi, log
from dataclasses import dataclass
from typing import Tuple
from numpy.linalg import lstsq

# Load your data: replace 'data.csv' or set df directly
try:
    df = pd.read_csv("data.csv")
except FileNotFoundError:
    # create a synthetic demo: sinusoid with slight kappa-phase drag + noise
    rng = np.random.default_rng(42)
    t = np.linspace(0, 10, 400)
    A_true, w_true, phi0_true, kappa_true = 1.0, 2.1, 0.2, 0.08
    y = A_true*np.sin(w_true*t + phi0_true + kappa_true*t) + 0.15*rng.normal(size=t.size)
    df = pd.DataFrame({"t": t, "y": y})

df.head()

## Models

- **Standard sine**: \( y(t) = A\sin(\omega t + \phi_0) + C \)
- **Helical sine**: \( y(t) = A\sin(\omega t + \phi_0 + \kappa t) + C \)

We fit by nonlinear least squares (simple grid + linearization) and compute **AIC/BIC**.

In [ ]:
from itertools import product

def fit_sine(t, y):
    # grid over omega, phi0; solve A,C linearly
    w_grid = np.linspace(0.5, 4.0, 60)
    phi_grid = np.linspace(-pi, pi, 60)
    best = None
    for w, phi0 in product(w_grid, phi_grid):
        X = np.column_stack([np.sin(w*t + phi0), np.ones_like(t)])
        coeff, *_ = lstsq(X, y, rcond=None)
        A, C = coeff
        yhat = X @ coeff
        rss = np.sum((y - yhat)**2)
        k = 3  # A, C, (w, phi0 counted as 2) -> total 4, but two were grid SLOW; use 4 for AIC/BIC fairness
        k = 4
        n = len(y)
        aic = n*np.log(rss/n) + 2*k
        bic = n*np.log(rss/n) + k*np.log(n)
        cand = (aic, bic, rss, A, w, phi0, C)
        if best is None or aic < best[0]:
            best = cand
    return best

def fit_helical(t, y):
    # grid over w, phi0, kappa; solve A,C linearly
    w_grid = np.linspace(0.5, 4.0, 40)
    phi_grid = np.linspace(-pi, pi, 40)
    kappa_grid = np.linspace(-0.3, 0.3, 31)
    best = None
    for w, phi0, kappa in product(w_grid, phi_grid, kappa_grid):
        phase = w*t + phi0 + kappa*t
        X = np.column_stack([np.sin(phase), np.ones_like(t)])
        coeff, *_ = lstsq(X, y, rcond=None)
        A, C = coeff
        yhat = X @ coeff
        rss = np.sum((y - yhat)**2)
        k = 5  # A, C, w, phi0, kappa
        n = len(y)
        aic = n*np.log(rss/n) + 2*k
        bic = n*np.log(rss/n) + k*np.log(n)
        cand = (aic, bic, rss, A, w, phi0, kappa, C)
        if best is None or aic < best[0]:
            best = cand
    return best

t = df["t"].values
y = df["y"].values

best_sine = fit_sine(t, y)         # (aic, bic, rss, A, w, phi0, C)
best_helix = fit_helical(t, y)     # (aic, bic, rss, A, w, phi0, kappa, C)

best_sine, best_helix

In [ ]:
# Report comparison
def report_models(best_sine, best_helix):
    aic_s, bic_s, rss_s, A_s, w_s, phi0_s, C_s = best_sine
    aic_h, bic_h, rss_h, A_h, w_h, phi0_h, kappa_h, C_h = best_helix
    return pd.DataFrame([
        {"model":"sine", "AIC":aic_s, "BIC":bic_s, "RSS":rss_s, "A":A_s, "w":w_s, "phi0":phi0_s, "kappa":"-","C":C_s},
        {"model":"helical", "AIC":aic_h, "BIC":bic_h, "RSS":rss_h, "A":A_h, "w":w_h, "phi0":phi0_h, "kappa":kappa_h,"C":C_h},
    ])

comparison = report_models(best_sine, best_helix)
comparison